In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("East_Massachusetts_Apr_PM");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/East_Massachusetts_trips_Apr_PM.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 64 entries:
  (3,6) => 1315.275222
  (4,4) => 0.0
  (3,1) => 302.447243
  (4,5) => 278.568958
  (2,4) => 459.387746
  (6,5) => 754.478522
  (8,4) => 383.709803
  (1,2) => 364.030563
  (5,1) => 374.332883
  (3,4) => 636.435156
  (8,2) => 234.682173
  (2,3) => 326.300096
  (2,6) => 663.680693
  (5,4) => 472.954265
  (2,1) => 511.457588
  (2,5) => 581.073471
  (8,5) => 402.014719
  (1,4) => 387.087581
  (2,8) => 213.569741
  (5,8) => 260.003198
  (4,8) => 127.833452
  (7,8) => 188.789153
  (1,1) => 0.0
  (7,7) => 0.0
  (5,7) => 600.509755
  ⋮     => ⋮

In [7]:
odPairLabel_

Dict{UTF8String,Any} with 56 entries:
  "32" => {5,4}
  "29" => {5,1}
  "1"  => {1,2}
  "54" => {8,5}
  "2"  => {1,3}
  "41" => {6,7}
  "53" => {8,4}
  "51" => {8,2}
  "27" => {4,7}
  "42" => {6,8}
  "33" => {5,6}
  "28" => {4,8}
  "52" => {8,3}
  "50" => {8,1}
  "26" => {4,6}
  "10" => {2,4}
  "24" => {4,3}
  "25" => {4,5}
  "23" => {4,2}
  "49" => {7,8}
  "5"  => {1,6}
  "31" => {5,3}
  "43" => {7,1}
  "39" => {6,4}
  "44" => {7,2}
  ⋮    => ⋮

In [8]:
demandsVec = zeros(56)

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

56-element Array{Float64,1}:
  364.031
  325.364
  387.088
  543.744
  531.093
  533.457
  187.046
  511.458
  326.3  
  459.388
  581.073
  663.681
  610.212
    ⋮    
  714.337
  517.884
  849.503
 1621.38 
  188.789
  239.612
  234.682
  514.833
  383.71 
  402.015
  566.538
  282.776

In [10]:
for key=keys(odPairRoute)
    if contains(key, "56-")
        println(key)
    end
end

56-314
56-313
56-312


In [11]:
linkRoute

Dict{UTF8String,Any} with 986 entries:
  "5-260"  => 1.0
  "3-290"  => 1.0
  "20-207" => 1.0
  "16-217" => 1.0
  "6-198"  => 1.0
  "19-180" => 1.0
  "23-180" => 1.0
  "16-48"  => 1.0
  "10-128" => 1.0
  "16-266" => 1.0
  "11-148" => 1.0
  "14-299" => 1.0
  "21-112" => 1.0
  "18-290" => 1.0
  "16-235" => 1.0
  "5-70"   => 1.0
  "9-93"   => 1.0
  "9-60"   => 1.0
  "1-14"   => 1.0
  "11-19"  => 1.0
  "17-190" => 1.0
  "11-52"  => 1.0
  "19-65"  => 1.0
  "10-291" => 1.0
  "12-241" => 1.0
  ⋮        => ⋮

In [12]:
coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM.json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["(8,0.5,10000.0,1)"]

9-element Array{Any,1}:
  1.0       
 -0.00302509
  0.0577279 
 -0.195632  
  0.620696  
 -0.905963  
  0.936143  
 -0.469483  
  0.108584  

In [13]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 24
numRoute = 314
numOD = 56

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
    free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
    free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6 +
    free_flow_time[a] * fcoeffs[8] * linkFlow[a]^8 / capacity[a]^7 +
    free_flow_time[a] * fcoeffs[9] * linkFlow[a]^9 / capacity[a]^8, a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1324
Number of nonzeros in inequality constraint Jacobian.:      314
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:      338
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [14]:
getValue(linkFlow)

24-element Array{Float64,1}:
 1785.96 
  701.911
 1085.87 
 1671.61 
 3429.67 
 1939.71 
  846.475
  354.302
 5207.62 
 4199.34 
 2998.45 
 1465.06 
 1189.39 
 2275.66 
 1139.88 
 1402.86 
  127.833
  383.71 
 3354.52 
 4345.24 
 1478.36 
 2258.64 
 1438.49 
 2240.46 

In [15]:
getObjectiveValue(m)

10868.839136251017

In [16]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 24 entries:
  (3,6) => 2998.4454658586897
  (2,3) => 3429.6727776218027
  (5,6) => 1139.8781390979877
  (5,4) => 2275.6583249228456
  (2,1) => 701.9114412529308
  (6,3) => 1465.0646073535527
  (3,1) => 1671.6113585734893
  (1,3) => 1085.866113305218
  (4,5) => 1189.3858859553195
  (4,8) => 127.8334519361988
  (7,5) => 4345.237365625287
  (8,7) => 2240.456486904579
  (3,2) => 1939.71163735912
  (7,6) => 2258.639957203024
  (2,4) => 846.4753267897162
  (7,8) => 1438.4946519314647
  (6,5) => 1402.8630651233193
  (6,7) => 1478.3587076828294
  (8,4) => 383.70980296308494
  (1,2) => 1785.955300521202
  (5,3) => 4199.336566582771
  (4,2) => 354.3024947841282
  (3,5) => 5207.623358072046
  (5,7) => 3354.5154791723685

In [17]:
# outfile = open("flows_MA.json", "w")

# JSON.print(outfile, flows)

# close(outfile)

In [18]:
# using JuMP

# # m = Model(solver=GurobiSolver(OutputFlag=false))
# m = Model()

# numLinks = 24  # number of links
# numNodes = 8  # number of nodes

# @defVar(m, linkFlow[1:numLinks])

# @defVar(m, odLinkFlow[keys(demands), 1:numLinks])
# for (s,t) = keys(demands)
#     for k = 1:numLinks
#         @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
#     end
# end

# for k = 1:numLinks
#     linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
#     @addConstraint(m, linkFlowPartial == linkFlow[k])
# end

# for l = 1:numNodes
#     for (s,t) = keys(demands)
#         if s != t
#             odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
#             if (l == s)
#                 @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
#             elseif (l == t)
#                 @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
#             else
#                 @addConstraint(m, odLinkFlowPartial == 0)
#             end
#         end
#     end
# end

# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:24})

# @setNLObjective(m, Min, f)

# solve(m)

In [19]:
# getValue(linkFlow)

In [20]:
# getObjectiveValue(m)

In [21]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...


In [22]:
flow_user = GLS_Apr_weekday_PM.x_

24x30 Array{Float64,2}:
 1468.48  2887.21  3396.09  3176.74  …  3436.13  2544.48  1991.71  1724.99
 1913.51  1785.06  1505.4   1768.44     1636.06  1674.9   1634.01  2242.21
 2685.38  1917.63  2168.52  2036.55     1933.07  1863.47  2171.32  2610.22
 3019.04  2648.3   2296.73  2541.17     2349.07  2168.59  2585.11  2093.1 
 2188.54  4130.96  6993.85  3314.72     2986.23  2045.14  2207.91  2656.21
 2800.07  2101.55  2008.23  1951.03  …  1636.08  2105.63  1520.69  1800.1 
 1638.15  1643.23  2740.69  2925.09     3023.63  1411.12  1591.81  2567.03
 2229.0   1943.95  1867.89  1935.72     1611.75  1428.49  1574.32  3034.69
 2998.65  3607.01  6300.37  6845.22     6218.79  2276.28  2968.94  6358.73
 2960.27  3864.1   3951.95  4929.88     4282.8   2452.26  2173.73  4574.88
 3332.75  5300.41  6927.29  5841.46  …  6566.89  3130.58  3202.07  6816.34
 3046.12  3410.43  3814.36  3684.83     3024.27  3278.96  2720.3   3215.74
 1482.62  1506.15  1348.81  1638.99     1312.12  2004.32  1415.54  1361.87
 

In [23]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 + free_flow_time[a] * fcoeffs[8] * linkFlowVec[a]^8 / capacity[a]^7 + free_flow_time[a] * fcoeffs[9] * linkFlowVec[a]^9 / capacity[a]^8 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [24]:
for i = 1:30
    println(socialObj(flow_user[:, i])/getObjectiveValue(m))
end

1.3987922991730621
1.4988399534013912
1.9076235871612943
1.8252039836603506
2.0899844547169195
1.690984989728792
1.201707438870099
1.3663769413021225
1.456054577399142
1.5701527047213173
1.748819864730797
2.244594276737557
1.8937484389390264
1.185769590967761
1.151661997622998
1.3255977529292693
1.6902987109876253
2.0640916320738194
1.9369571379172823
1.7764050106517995
1.1146733290689883
1.5383898151295183
1.4722226439117234
1.66495498789451
1.7547107631122831
1.994322535468672
1.845239088010564
1.248219346336131
1.1757442564932292
1.7308690710221917
